# Tổng quan: Phát hiện đối tượng với Faster R-CNN

## Mục tiêu
Phát triển một mô hình phát hiện đối tượng để vẽ **bounding box** xung quanh các đối tượng trong ảnh và gán nhãn tương ứng. Trong bài toán này, thuật toán **Faster R-CNN** được sử dụng với các thành phần chính:

1. **Mạng Backbone**: Học các đặc trưng (features) của ảnh.
2. **Region Proposal Network (RPN)**: Tìm các anchor ứng viên, đóng vai trò là đầu vào cho bước tiếp theo.
3. **RoI Pooling**: Kết hợp đầu ra từ RPN và các đặc trưng của ảnh để xác định bounding box và nhãn tương ứng.

### Kiến trúc Faster R-CNN
Dưới đây là sơ đồ minh họa kiến trúc của Faster R-CNN:

![Faster R-CNN Architecture](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRW1T3Csvlu2VxcPI_KW_d5e-_5aPG-QLEwg&s)

---

## Dữ liệu
- **Tổng số ảnh**: 7600 ảnh.
- **Số nhãn**: 6 nhãn (multi-class object detection).
- **Kích thước ảnh**: 640x640 pixel.
- **Phân chia dữ liệu**:
  - **Train**: Toàn bộ tập ảnh dùng để huấn luyện mô hình.
  - **Test**: 20 ảnh được lựa chọn (kiểm tra thủ công bằng mắt thường để đánh giá chất lượng).

---

## Phương pháp
Xây dựng mô hình theo cách tiếp cận "từ dưới lên trên," tức là đi từ từng thành phần cơ bản của thuật toán (RPN, Backbone, RoI Pooling) đến hệ thống hoàn chỉnh.


# Xây dựng mô hình Faster R-CNN

## 1. Xây dựng mô hình ResNet18
Mục tiêu là thiết lập mô hình trích xuất đặc trưng của ảnh. Trong bài toán này, sử dụng **ResNet18** làm mạng backbone. 

### Cách thực hiện:
- Sử dụng mô hình **ResNet18** đã được huấn luyện trước (pre-trained).
- Loại bỏ các lớp cuối cùng:
  - **Lớp avgpool** (average pooling).
  - **Lớp Linear** (fully connected layer).

### Đầu vào và đầu ra:
- **Input**: 
  - Kích thước: `(batch_size=4, channels=3, height=640, width=640)` 
  - Tương ứng với batch gồm 4 ảnh, mỗi ảnh có 3 kênh màu RGB, kích thước 640x640.
- **Output**: 
  - Kích thước: `(batch_size=4, channels=512, height=20, width=20)`
  - Gồm 512 kênh đặc trưng, với mỗi kênh có kích thước 20x20.

Kết quả đầu ra này sẽ được sử dụng làm input cho các bước tiếp theo trong pipeline của Faster R-CNN.

In [1]:
import torchvision.models as models
import torch.nn as nn
import torch
import math
import torchvision

class BackBoneResNet18(nn.Module):
    def __init__(self):
        super().__init__()

        weights = models.ResNet18_Weights.DEFAULT
        resnet18 = models.resnet18(weights=weights)

        children = [child for child in resnet18.children()]

        self.model = nn.Sequential(*children[:-2])
        # self.model = resnet18.features[:-1]

    def forward(self, X):

        out = self.model(X)

        return out


## 2. Xây dựng Region Proposal Network (RPN)

### Định nghĩa một số hàm hỗ trợ trong RPN

#### Hàm `get_iou`
Hàm này tính tỷ lệ chồng lấp (IoU - Intersection over Union) giữa các anchor. Đây là một bước quan trọng trong việc xác định chất lượng của các anchor được đề xuất bởi RPN.

### Công thức:
$$
IoU = \frac{\text{Diện tích phần chồng lấp (overlap)}}{\text{Diện tích hợp nhất (union)}}
$$

### Ý nghĩa:
- **Diện tích chồng lấp (overlap)**: Phần giao nhau giữa hai anchor trên mặt phẳng.
- **Diện tích hợp nhất (union)**: Tổng diện tích của hai anchor trừ đi phần diện tích giao nhau.

### Ứng dụng:
- Dùng để đánh giá các anchor:
  - Anchor nào tốt (có IoU cao) sẽ được chọn làm ứng viên cho bước tiếp theo.
  - Anchor nào có IoU thấp sẽ bị loại bỏ.

Hàm `get_iou` sẽ là một phần quan trọng trong quá trình sinh ra các anchor và chọn lọc chúng trong pipeline của Region Proposal Network.


In [2]:
def get_iou(boxes1, boxes2):
    r"""
    IOU between two sets of boxes
    :param boxes1: (Tensor of shape N x 4)
    :param boxes2: (Tensor of shape M x 4)
    :return: IOU matrix of shape N x M
    """
    # Area of boxes (x2-x1)*(y2-y1)
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])  # (N,)
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])  # (M,)
    
    # Get top left x1,y1 coordinate
    x_left = torch.max(boxes1[:, None, 0], boxes2[:, 0])  # (N, M)
    y_top = torch.max(boxes1[:, None, 1], boxes2[:, 1])  # (N, M)
    
    # Get bottom right x2,y2 coordinate
    x_right = torch.min(boxes1[:, None, 2], boxes2[:, 2])  # (N, M)
    y_bottom = torch.min(boxes1[:, None, 3], boxes2[:, 3])  # (N, M)
    
    intersection_area = (x_right - x_left).clamp(min=0) * (y_bottom - y_top).clamp(min=0)  # (N, M)
    union = area1[:, None] + area2 - intersection_area  # (N, M)
    iou = intersection_area / union  # (N, M)
    return iou

#### Hàm `clamp_boxes_to_image_boundary`

Hàm này đảm bảo rằng các tọa độ của các hộp giới hạn (bounding boxes) nằm hoàn toàn trong biên giới của hình ảnh. Nếu các hộp vượt ra ngoài phạm vi hình ảnh, các tọa độ sẽ được điều chỉnh sao cho phù hợp với kích thước thực tế của hình ảnh.

### Đầu vào:
- **boxes**: Tensor có dạng `(N, 4)`, với mỗi hàng biểu diễn một hộp giới hạn dưới dạng `(x_min, y_min, x_max, y_max)`.
- **image_shape**: Kích thước của hình ảnh dưới dạng tuple `(height, width)`.

### Đầu ra:
- **boxes**: Tensor với các tọa độ đã được điều chỉnh sao cho chúng nằm trong biên giới của hình ảnh.

### Mô tả:
- Hàm này điều chỉnh các giá trị của `x_min`, `y_min`, `x_max`, `y_max` sao cho chúng không vượt qua phạm vi của hình ảnh. Các giá trị sẽ được giới hạn trong phạm vi từ 0 đến chiều cao (height) và chiều rộng (width) của hình ảnh.


In [3]:
def clamp_boxes_to_image_boundary(boxes, image_shape):
    boxes_x1 = boxes[..., 0]
    boxes_y1 = boxes[..., 1]
    boxes_x2 = boxes[..., 2]
    boxes_y2 = boxes[..., 3]
    height, width = image_shape[-2:]
    boxes_x1 = boxes_x1.clamp(min=0, max=width)
    boxes_x2 = boxes_x2.clamp(min=0, max=width)
    boxes_y1 = boxes_y1.clamp(min=0, max=height)
    boxes_y2 = boxes_y2.clamp(min=0, max=height)
    boxes = torch.cat((
        boxes_x1[..., None],
        boxes_y1[..., None],
        boxes_x2[..., None],
        boxes_y2[..., None]),
        dim=-1)
    return boxes

#### Hàm `apply_regression_pred_to_anchors_or_proposals`

**Mục đích**:  
Hàm này có nhiệm vụ biến đổi các thông số dự đoán từ mô hình (dưới dạng `dx`, `dy`, `dw`, `dh`) cho các anchor hoặc proposal, thành các hộp dự đoán (predicted boxes).

### Đầu vào:
- **anchors** (hoặc **proposals**): Tensor có dạng `(N, 4)`, trong đó mỗi hàng biểu diễn một hộp giới hạn (anchor hoặc proposal) dưới dạng `(x_min, y_min, x_max, y_max)`.
- **predictions**: Tensor có dạng `(N, 4)`, với mỗi hàng chứa các thông số dự đoán từ mô hình dưới dạng `(dx, dy, dw, dh)`.

### Đầu ra:
- **predicted_boxes**: Tensor có dạng `(N, 4)`, là các hộp giới hạn dự đoán (predicted boxes) được tính từ các thông số dự đoán và các anchor hoặc proposal ban đầu.

### Mô tả:
- Hàm này sử dụng các thông số dự đoán `dx`, `dy`, `dw`, `dh` để điều chỉnh các giá trị của các anchor hoặc proposal ban đầu, từ đó tạo ra các hộp giới hạn dự đoán. Quá trình này bao gồm việc áp dụng các phép toán hồi quy lên tọa độ của anchor hoặc proposal để tạo ra các predicted boxes.


In [4]:
def apply_regression_pred_to_anchors_or_proposals(box_transform_pred, anchors_or_proposals):
    r"""
    Dựa trên dự đoán các tham số biến đổi box cho tất cả các anchor hoặc proposal đầu vào,
    biến đổi chúng tương ứng để tạo ra các proposal hoặc box dự đoán.

    :param box_transform_pred: Tensor có kích thước (num_anchors_or_proposals, num_classes, 4),
                               chứa các dự đoán thay đổi box (dx, dy, dw, dh) cho từng anchor hoặc proposal.
    :param anchors_or_proposals: Tensor có kích thước (num_anchors_or_proposals, 4),
                                 chứa các giá trị của các anchor hoặc proposal dưới dạng (x1, y1, x2, y2).
    :return pred_boxes: Tensor có kích thước (num_anchors_or_proposals, num_classes, 4),
                        chứa các box dự đoán cho từng lớp (x1, y1, x2, y2).
    """
    
    # Biến đổi lại kích thước của box_transform_pred
    box_transform_pred = box_transform_pred.reshape(
        box_transform_pred.size(0), -1, 4)

    # Tính toán chiều rộng (w) và chiều cao (h) từ x1, y1, x2, y2 của anchor hoặc proposal
    w = anchors_or_proposals[:, 2] - anchors_or_proposals[:, 0]
    h = anchors_or_proposals[:, 3] - anchors_or_proposals[:, 1]
    center_x = anchors_or_proposals[:, 0] + 0.5 * w
    center_y = anchors_or_proposals[:, 1] + 0.5 * h
    
    # Lấy các giá trị dx, dy, dw, dh từ dự đoán
    dx = box_transform_pred[..., 0]
    dy = box_transform_pred[..., 1]
    dw = box_transform_pred[..., 2]
    dh = box_transform_pred[..., 3]
    
    # Giới hạn giá trị dw, dh để tránh đưa quá giá trị lớn vào hàm torch.exp()
    dw = torch.clamp(dw, max=math.log(1000.0 / 16))
    dh = torch.clamp(dh, max=math.log(1000.0 / 16))
    
    # Tính toán các giá trị trung tâm và kích thước của hộp dự đoán
    pred_center_x = dx * w[:, None] + center_x[:, None]
    pred_center_y = dy * h[:, None] + center_y[:, None]
    pred_w = torch.exp(dw) * w[:, None]
    pred_h = torch.exp(dh) * h[:, None]
    
    # Tính toán các tọa độ của hộp dự đoán (x1, y1, x2, y2)
    pred_box_x1 = pred_center_x - 0.5 * pred_w
    pred_box_y1 = pred_center_y - 0.5 * pred_h
    pred_box_x2 = pred_center_x + 0.5 * pred_w
    pred_box_y2 = pred_center_y + 0.5 * pred_h
    
    # Kết hợp các tọa độ của hộp dự đoán vào một tensor duy nhất
    pred_boxes = torch.stack((
        pred_box_x1,
        pred_box_y1,
        pred_box_x2,
        pred_box_y2),
        dim=2)
    
    return pred_boxes

#### Hàm `boxes_to_transformation_targets`

**Mục đích**:  
Hàm này chuyển đổi các tọa độ hộp (x1, y1, x2, y2) của các hộp ground truth và các anchor/proposal thành các giá trị biến đổi (tx, ty, tw, th) mà mô hình có thể sử dụng trong quá trình huấn luyện.

### Cách tính các giá trị biến đổi:
1. **tx: Sự thay đổi trung tâm theo trục x:**
$$
\text{targets\_dx} = \frac{\text{gt\_center\_x} - \text{center\_x}}{\text{widths}}
$$

2. **ty: Sự thay đổi trung tâm theo trục y:**
$$
\text{targets\_dy} = \frac{\text{gt\_center\_y} - \text{center\_y}}{\text{heights}}
$$

3. **tw: Sự thay đổi chiều rộng:**
$$
\text{targets\_dw} = \log \left( \frac{\text{gt\_widths}}{\text{widths}} \right)
$$

4. **th: Sự thay đổi chiều cao:**
$$
\text{targets\_dh} = \log \left( \frac{\text{gt\_heights}}{\text{heights}} \right)
$$

### Đầu vào:
- **gt_boxes**: Tensor có dạng `(N, 4)`, với mỗi hàng là các tọa độ của hộp ground truth dưới dạng `(x1, y1, x2, y2)`.
- **anchors**: Tensor có dạng `(N, 4)`, với mỗi hàng là các tọa độ của anchor/proposal dưới dạng `(x1, y1, x2, y2)`.

### Đầu ra:
- **tx, ty, tw, th**: Các giá trị biến đổi (dx, dy, dw, dh) được tính cho từng anchor/proposal so với hộp ground truth.

### Mô tả:
Hàm này giúp tính toán các giá trị biến đổi để huấn luyện mô hình phát hiện đối tượng. Các giá trị biến đổi này giúp mô hình có thể học cách chuyển đổi từ các anchor hoặc proposal ban đầu thành các hộp giới hạn chính xác hơn thông qua quá trình hồi quy trong quá trình huấn luyện.


In [5]:
def boxes_to_transformation_targets(ground_truth_boxes, anchors_or_proposals):
    r"""
    Dựa trên các hộp ground truth và các anchor/proposal trong hình ảnh, hàm này tính toán
    các giá trị tx, ty, tw, th (biến đổi tọa độ hộp) cho tất cả các anchor hoặc proposal.
    
    :param ground_truth_boxes: Tensor có kích thước (anchors_or_proposals_in_image, 4),
                                chứa các giá trị tọa độ (x1, y1, x2, y2) của các hộp ground truth.
    :param anchors_or_proposals: Tensor có kích thước (anchors_or_proposals_in_image, 4),
                                 chứa các giá trị tọa độ (x1, y1, x2, y2) của các anchor hoặc proposal.
    :return: regression_targets: Tensor có kích thước (anchors_or_proposals_in_image, 4),
                                 chứa các giá trị tx, ty, tw, th (biến đổi tọa độ hộp) cho tất cả
                                 các anchor/proposal.
    """
    
    # Tính toán center_x, center_y, w, h từ x1, y1, x2, y2 cho các anchors
    widths = anchors_or_proposals[:, 2] - anchors_or_proposals[:, 0]
    heights = anchors_or_proposals[:, 3] - anchors_or_proposals[:, 1]
    center_x = anchors_or_proposals[:, 0] + 0.5 * widths
    center_y = anchors_or_proposals[:, 1] + 0.5 * heights
    
    # Tính toán center_x, center_y, w, h từ x1, y1, x2, y2 cho các hộp ground truth
    gt_widths = ground_truth_boxes[:, 2] - ground_truth_boxes[:, 0]
    gt_heights = ground_truth_boxes[:, 3] - ground_truth_boxes[:, 1]
    gt_center_x = ground_truth_boxes[:, 0] + 0.5 * gt_widths
    gt_center_y = ground_truth_boxes[:, 1] + 0.5 * gt_heights
    
    # Tính toán các giá trị tx, ty, tw, th từ sự khác biệt giữa ground truth và anchor/proposal
    targets_dx = (gt_center_x - center_x) / widths
    targets_dy = (gt_center_y - center_y) / heights
    targets_dw = torch.log(gt_widths / widths)
    targets_dh = torch.log(gt_heights / heights)
    
    # Gộp các giá trị tx, ty, tw, th thành một tensor
    regression_targets = torch.stack((targets_dx, targets_dy, targets_dw, targets_dh), dim=1)
    
    return regression_targets

#### Hàm `sample_positive_negative`

**Mục đích**:  
Hàm này chọn ngẫu nhiên một số lượng cụ thể các anchor box "positive" và "negative" từ các anchor box có sẵn, tạo ra các mặt nạ để xác định những anchor box này. Các anchor box được chọn sẽ được chuẩn bị cho quá trình huấn luyện của Region Proposal Network (RPN), giúp cân bằng giữa các anchor hợp lệ (chứa object) và các anchor không hợp lệ (không chứa object).

### Cách thức hoạt động:
1. **Positive Anchors**:  
   Các anchor box được gắn nhãn "positive" nếu chúng có tỉ lệ IoU (Intersection over Union) lớn hơn một ngưỡng nhất định với các hộp ground truth.

2. **Negative Anchors**:  
   Các anchor box được gắn nhãn "negative" nếu chúng có tỉ lệ IoU nhỏ hơn một ngưỡng nhất định với các hộp ground truth.

3. **Chọn ngẫu nhiên**:  
   Sau khi phân loại các anchor box thành "positive" và "negative", hàm sẽ chọn một số lượng ngẫu nhiên các anchor box từ mỗi loại, nhằm đảm bảo sự cân bằng giữa các anchor hợp lệ và không hợp lệ.

### Mô tả:
Hàm `sample_positive_negative` giúp xác định các anchor box nào sẽ được sử dụng trong quá trình huấn luyện để tối ưu hóa mô hình. Việc chọn các anchor "positive" và "negative" giúp mô hình học cách phân biệt giữa các anchor chứa đối tượng và các anchor không chứa đối tượng, từ đó cải thiện độ chính xác của Region Proposal Network (RPN).


In [6]:
def sample_positive_negative(labels, positive_count, total_count):
    # Sample positive và negative proposals
    positive = torch.where(labels >= 1)[0]
    negative = torch.where(labels == 0)[0]
    num_pos = positive_count
    num_pos = min(positive.numel(), num_pos)
    num_neg = total_count - num_pos
    num_neg = min(negative.numel(), num_neg)
    perm_positive_idxs = torch.randperm(positive.numel(),
                                        device=positive.device)[:num_pos]
    perm_negative_idxs = torch.randperm(negative.numel(),
                                        device=negative.device)[:num_neg]
    pos_idxs = positive[perm_positive_idxs]
    neg_idxs = negative[perm_negative_idxs]
    sampled_pos_idx_mask = torch.zeros_like(labels, dtype=torch.bool)
    sampled_neg_idx_mask = torch.zeros_like(labels, dtype=torch.bool)
    sampled_pos_idx_mask[pos_idxs] = True
    sampled_neg_idx_mask[neg_idxs] = True
    return sampled_neg_idx_mask, sampled_pos_idx_mask


## 3. Xây dựng Region Proposal Network (RPN)

Tiếp theo, em sẽ xây dựng **Region Proposal Network (RPN)**, một phần quan trọng trong Faster R-CNN. RPN về cơ bản là một mạng sinh các anchor tại mỗi điểm của feature map, sau đó sử dụng các thuật toán như **Non-maximum Suppression (NMS)** để lọc ra top K các anchor tốt nhất.

#### Các phương thức trong `RegionProposalNetwork`:

1. **`generate_anchor`**:
   - Mục đích: Tạo ra các anchor tại mỗi điểm trên feature map.
   - Đầu vào:
     - **image** (ban đầu) và **feature map**.
   - Đầu ra: Tại mỗi vị trí trên feature map, có **K=9 anchors** với thông tin `(x, y)` là tọa độ tâm của anchor và `(h, w)` là kích thước của anchor.

2. **`assign_targets_to_anchors`**:
   - Mục đích: Gán nhãn và tọa độ cho từng anchor.
   - Mỗi anchor sẽ được gán nhãn dựa trên tỉ lệ **Intersection over Union (IoU)** với ground truth box:
     - **Foreground (label = 1)**: Anchor có IoU với ground truth box lớn hơn ngưỡng **high_iou_threshold**.
     - **Background (label = 0)**: Anchor có IoU nhỏ hơn ngưỡng **low_iou_threshold**.
     - **Ignored (label = -1)**: Anchor có IoU nằm giữa hai ngưỡng và bị bỏ qua trong huấn luyện.
   - Đảm bảo mỗi ground truth box có ít nhất một anchor chất lượng cao được gán.

3. **`filter_proposals`**:
   - Mục đích: Sử dụng thuật toán **Non-Maximum Suppression (NMS)** để lọc ra **K proposal anchors** tốt nhất.

4. **`forward`**:
   - Các bước trong quá trình forward:
     1. Gọi các lớp RPN để tạo dự đoán phân loại (classification) và biến đổi hộp giới hạn (bbox transformation) cho các anchors.
     2. Sinh các anchors cho toàn bộ hình ảnh.
     3. Biến đổi các anchors dựa trên dự đoán biến đổi hộp giới hạn để tạo ra các proposals.
     4. Lọc các proposals.
     5. Trong quá trình huấn luyện:
        - Gán nhãn mục tiêu và hộp giới hạn ground truth cho từng anchor.
        - Lấy mẫu các anchors dương (positive) và âm (negative).
        - Tính toán tổn thất phân loại sử dụng các anchors dương/âm.
        - Tính toán tổn thất định vị (localization loss) sử dụng các anchors dương.

#### Mạng RPN:
- **RPN conv layer**: Lớp convolution này không thay đổi số lượng kênh đầu vào nhưng giúp mô hình học được các đặc trưng cần thiết cho phân loại và hồi quy.
- **Cls layer**: Lớp này thực hiện phân loại cho từng anchor trong feature map.
- **BBox regression layer**: Dự đoán các tọa độ mới `(x1, y1, x2, y2)` cho mỗi anchor.

### Loss function của RPN:
Mục tiêu của RPN là học cách phân loại và điều chỉnh các anchor boxes sao cho chúng có thể khớp tốt với các đối tượng trong ảnh. Tổng tổn thất **RPN loss** là sự kết hợp của hai thành phần:

1. **Classification Loss**: Tổn thất phân loại xác định xem anchor là foreground (1) hay background (0). Sử dụng **Binary Cross-Entropy Loss** cho phân loại.
   
   $$
   L_{cls} = - \frac{1}{N_{cls}} \sum_{i} \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
   $$

   Trong đó:
   - $ y_i $ là nhãn của anchor $ i $ (1 cho foreground, 0 cho background).
   - $ p_i $ là xác suất mà anchor $ i $ được phân loại là foreground.

2. **Bounding Box Regression Loss**: Tổn thất hồi quy xác định sự thay đổi giữa các anchor box và các ground truth box. Sử dụng **Smooth L1 Loss** cho tổn thất này.
   
   $$
   L_{bbox} = \frac{1}{N_{reg}} \sum_{i} \text{SmoothL1}(t_i - \hat{t}_i)
   $$

   Trong đó:
   - $ t_i $ là biến đổi hộp giới hạn cho anchor $i$.
   - $\hat{t}_i $ là dự đoán biến đổi hộp giới hạn từ mô hình.
   - **Smooth L1 Loss** được tính như sau:

   $$
   \text{SmoothL1}(x) =
   \begin{cases}
   0.5x^2, & \text{nếu } |x| < 1 \\
   |x| - 0.5, & \text{nếu } |x| \geq 1
   \end{cases}
   $$

3. **Tổng tổn thất (RPN Loss)**: Tổng tổn thất là sự kết hợp của hai thành phần trên:
   
   $$
   L_{rpn} = L_{cls} + \lambda L_{bbox}
   $$
   
   Trong đó:
   - $\lambda$ là một hệ số cân bằng giữa các thành phần phân loại và hồi quy.

Tổn thất này sẽ được tối ưu hóa trong quá trình huấn luyện để cải thiện khả năng phát hiện đối tượng của RPN.


In [7]:
class RegionProposalNetwork(nn.Module):
    r"""
    RPN với các lớp sau trên feature map
        1. Lớp đối lưu 3x3 theo sau là Relu
        2. Chuyển đổi phân loại 1x1 với các kênh đầu ra num_anchors(num_scales x num_aspect_ratios)
        3. Chuyển đổi phân loại 1x1 với 4 kênh đầu ra num_anchors

    Việc phân loại được thực hiện thông qua một giá trị biểu thị xác suất của foreground
    với sigmoid được áp dụng trong quá trình inference
    """
    
    def __init__(self, in_channels, scales, aspect_ratios, model_config):
        super(RegionProposalNetwork, self).__init__()
        self.scales = scales
        self.low_iou_threshold = model_config['rpn_bg_threshold']
        self.high_iou_threshold = model_config['rpn_fg_threshold']
        self.rpn_nms_threshold = model_config['rpn_nms_threshold']
        self.rpn_batch_size = model_config['rpn_batch_size']
        self.rpn_pos_count = int(model_config['rpn_pos_fraction'] * self.rpn_batch_size)
        self.rpn_topk = model_config['rpn_train_topk'] if self.training else model_config['rpn_test_topk']
        self.rpn_prenms_topk = model_config['rpn_train_prenms_topk'] if self.training \
            else model_config['rpn_test_prenms_topk']
        self.aspect_ratios = aspect_ratios
        self.num_anchors = len(self.scales) * len(self.aspect_ratios)
        
        # 3x3 conv layer
        self.rpn_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        
        # 1x1 classification conv layer
        self.cls_layer = nn.Conv2d(in_channels, self.num_anchors, kernel_size=1, stride=1)
        
        # 1x1 regression
        self.bbox_reg_layer = nn.Conv2d(in_channels, self.num_anchors * 4, kernel_size=1, stride=1)
        
        for layer in [self.rpn_conv, self.cls_layer, self.bbox_reg_layer]:
            torch.nn.init.normal_(layer.weight, std=0.01)
            torch.nn.init.constant_(layer.bias, 0)
    
    def generate_anchors(self, image, feat): # pass the test
        r"""
        Phương thức để tạo neo. Đầu tiên chúng ta tạo ra một tập hợp các neo không có trọng tâm
        bằng cách sử dụng thang đo và tỷ lệ khung hình được cung cấp.
        Sau đó, tạo các giá trị dịch chuyển theo trục x, y cho tất cả các vị trí feature map.
        Các điểm neo ở giữa bằng 0 được tạo ra sẽ được sao chép và dịch chuyển tương ứng
        để tạo điểm neo cho tất cả các vị trí bản đồ đặc trưng.
        Lưu ý rằng các điểm neo này được tạo sao cho tâm của chúng nằm ở góc trên bên trái của
        ô bản đồ đặc trưng chứ không phải là trung tâm của ô feature map.
        :param ảnh: tensor (N, C, H, W)
        :param feat: (N, C_feat, H_feat, W_feat) tensor
        :return: (H_feat * W_feat * num_anchors_per_location, 4)
        """
        grid_h, grid_w = feat.shape[-2:]
        image_h, image_w = image.shape[-2:]

        # Calculate stride values for height and width
        stride_h = (image_h // grid_h)
        stride_w = (image_w // grid_w)

        scales = torch.tensor(self.scales, dtype=feat.dtype, device=feat.device)
        aspect_ratios = torch.tensor(self.aspect_ratios, dtype=feat.dtype, device=feat.device)

        # Compute height and width ratios based on aspect ratios
        h_ratios = torch.sqrt(aspect_ratios)
        w_ratios = 1 / h_ratios

        # Calculate anchor box widths and heights based on scales and aspect ratios
        ws = (w_ratios[:, None] * scales[None, :]).flatten()
        hs = (h_ratios[:, None] * scales[None, :]).flatten()

        # Create base anchors centered at the origin
        base_anchors = torch.stack([-ws, -hs, ws, hs], dim=1) / 2
        base_anchors = base_anchors.round()

        # Compute shifts for x and y axes based on the feature map grid
        shifts_x = torch.arange(0, grid_w, dtype=torch.int32, device=feat.device) * stride_w
        shifts_y = torch.arange(0, grid_h, dtype=torch.int32, device=feat.device) * stride_h

        # Generate a meshgrid of shifts
        shifts_y, shifts_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")

        # Flatten the shifts
        shifts = torch.stack((shifts_x.flatten(), shifts_y.flatten(),
                            shifts_x.flatten(), shifts_y.flatten()), dim=1)

        # Add shifts to base anchors to create all anchor boxes
        anchors = (shifts[:, None, :] + base_anchors[None, :, :]).reshape(-1, 4)

        return anchors
    
    def assign_targets_to_anchors(self, anchors, gt_boxes):
        r"""
        Gán mỗi anchor với một ground truth box dựa trên IOU (Intersection Over Union).
        Đồng thời tạo nhãn phân loại để sử dụng trong quá trình huấn luyện:
        - label = 1: Với các anchor có IOU lớn hơn ngưỡng `high_iou_threshold`.
        - label = 0: Với các anchor có IOU nhỏ hơn ngưỡng `low_iou_threshold`.
        - label = -1: Với các anchor có IOU nằm trong khoảng giữa (`low_iou_threshold`, `high_iou_threshold`).

        :param anchors: Tensor có kích thước (num_anchors_in_image, 4) chứa tất cả anchor boxes.
        :param gt_boxes: Tensor có kích thước (num_gt_boxes_in_image, 4) chứa tất cả ground truth boxes.
        :return:
            - labels: Tensor (num_anchors_in_image) với giá trị {-1/0/1}.
            - matched_gt_boxes: Tensor (num_anchors_in_image, 4), tọa độ của ground truth box gán cho từng anchor.
                Ngay cả các anchor thuộc background hoặc bị bỏ qua cũng được gán với một ground truth box.
                Điều này không gây vấn đề vì nhãn sẽ phân biệt chúng sau này.
        """
        # Tính ma trận IOU giữa gt_boxes và anchors
        iou_matrix = get_iou(gt_boxes, anchors)

        # Lấy IOU lớn nhất và chỉ số gt_box tương ứng cho mỗi anchor
        best_match_iou, best_match_gt_idx = iou_matrix.max(dim=0)
        
        # Sao chép chỉ số ban đầu của gt_boxes để sử dụng sau này
        best_match_gt_idx_pre_thresholding = best_match_gt_idx.clone()

        # Cập nhật giá trị của best_match_gt_idx dựa trên ngưỡng IOU
        below_low_threshold = best_match_iou < self.low_iou_threshold
        between_thresholds = (best_match_iou >= self.low_iou_threshold) & (best_match_iou < self.high_iou_threshold)
        best_match_gt_idx[below_low_threshold] = -1  # Background
        best_match_gt_idx[between_thresholds] = -2  # Bị bỏ qua

        # Đảm bảo các ground truth boxes có ít nhất một anchor chất lượng thấp được gán
        best_anchor_iou_for_gt, _ = iou_matrix.max(dim=1)
        gt_pred_pair_with_highest_iou = torch.where(iou_matrix == best_anchor_iou_for_gt[:, None])
        pred_inds_to_update = gt_pred_pair_with_highest_iou[1]
        best_match_gt_idx[pred_inds_to_update] = best_match_gt_idx_pre_thresholding[pred_inds_to_update]

        # Chuyển đổi chỉ số gt_box để hợp lệ
        matched_gt_boxes = gt_boxes[best_match_gt_idx.clamp(min=0)]

        # Tạo nhãn phân loại
        labels = torch.full_like(best_match_gt_idx, fill_value=-1, dtype=torch.float32)
        labels[best_match_gt_idx >= 0] = 1.0  # Foreground
        labels[best_match_gt_idx == -1] = 0.0  # Background

        return labels, matched_gt_boxes

    def filter_proposals(self, proposals, cls_scores, image_shape):
        r"""
        Hàm này thực hiện các bước lọc và điều chỉnh danh sách đề xuất (proposals) nhằm tối ưu hóa đầu vào 
        cho các bước tiếp theo trong mạng phát hiện đối tượng.

        Các bước thực hiện bao gồm:
        1. **Lọc top K proposals trước NMS** (Pre-NMS topK filtering): Chọn ra các proposals có điểm số cao nhất.
        2. **Điều chỉnh tọa độ proposals hợp lệ**: Đảm bảo rằng tất cả các proposals nằm trong giới hạn hình ảnh.
        3. **Loại bỏ các hộp nhỏ**: Loại bỏ những proposals có chiều rộng hoặc chiều cao nhỏ hơn kích thước tối thiểu.
        4. **NMS (Non-Maximum Suppression)**: Loại bỏ các proposals trùng lặp dựa trên điểm số và ngưỡng IOU.
        5. **Lọc top K proposals sau NMS** (Post-NMS topK filtering): Giữ lại số lượng proposals tốt nhất sau NMS.

        :param proposals: Tensor (num_anchors_in_image, 4), tọa độ của các proposals (x_min, y_min, x_max, y_max).
        :param cls_scores: Tensor (num_anchors_in_image), điểm số dự đoán cho từng proposal (logits).
        :param image_shape: Tuple (height, width) của hình ảnh đã được resize, dùng để giới hạn tọa độ proposals.
        :return: 
            - **proposals**: Tensor (num_filtered_proposals, 4), các proposals sau khi lọc.
            - **cls_scores**: Tensor (num_filtered_proposals), điểm số tương ứng với các proposals đã lọc.
        """
        # Bước 1: Lọc top K proposals trước NMS
        cls_scores = cls_scores.reshape(-1)
        cls_scores = torch.sigmoid(cls_scores)
        _, top_n_idx = cls_scores.topk(min(self.rpn_prenms_topk, len(cls_scores)))
        cls_scores = cls_scores[top_n_idx]
        proposals = proposals[top_n_idx]
        
        # Bước 2: Giới hạn các proposals trong biên hình ảnh
        proposals = clamp_boxes_to_image_boundary(proposals, image_shape)
        
        # Bước 3: Lọc bỏ các hộp nhỏ
        min_size = 16  # Kích thước tối thiểu
        ws, hs = proposals[:, 2] - proposals[:, 0], proposals[:, 3] - proposals[:, 1]
        keep = (ws >= min_size) & (hs >= min_size)
        keep = torch.where(keep)[0]
        proposals = proposals[keep]
        cls_scores = cls_scores[keep]
        
        # Bước 4: Áp dụng NMS (Non-Maximum Suppression)
        keep_mask = torch.zeros_like(cls_scores, dtype=torch.bool)
        keep_indices = torch.ops.torchvision.nms(proposals, cls_scores, self.rpn_nms_threshold)
        keep_mask[keep_indices] = True
        keep_indices = torch.where(keep_mask)[0]
        
        # Sắp xếp lại theo điểm số objectness (điểm số cao nhất trước)
        post_nms_keep_indices = keep_indices[cls_scores[keep_indices].sort(descending=True)[1]]
        
        # Bước 5: Lọc top K proposals sau NMS
        proposals, cls_scores = (
            proposals[post_nms_keep_indices[:self.rpn_topk]],
            cls_scores[post_nms_keep_indices[:self.rpn_topk]],
        )
        
        return proposals, cls_scores

    
    def forward(self, image, feat, target=None):
        """
        Phương pháp chính cho RPN (Region Proposal Network) thực hiện các bước sau:
        1. Gọi các lớp RPN cụ thể để tạo dự đoán phân loại (classification) và
        biến đổi hộp giới hạn (bbox transformation) cho các anchors.
        2. Sinh các anchors cho toàn bộ hình ảnh.
        3. Biến đổi các anchors dựa trên dự đoán biến đổi hộp giới hạn để tạo ra các proposals.
        4. Lọc các proposals.
        5. Trong quá trình huấn luyện, thực hiện thêm các bước:
            a. Gán nhãn mục tiêu và hộp giới hạn ground truth cho từng anchor.
            b. Lấy mẫu các anchors dương (positive) và âm (negative).
            c. Tính toán tổn thất phân loại sử dụng các anchors dương/âm được lấy mẫu.
            d. Tính toán tổn thất định vị (localization loss) sử dụng các anchors dương.

        Tham số:
        - image: Tensor biểu diễn hình ảnh đầu vào.
        - feat: Tensor đặc trưng đầu vào từ backbone.
        - target: (Tùy chọn) Ground truth bao gồm nhãn và hộp giới hạn.

        Trả về:
        - rpn_output: Dictionary chứa các proposals, điểm số (scores), và mất mát (nếu có).
        """
        # Bước 1: Tính toán các đặc trưng RPN
        rpn_feat = nn.ReLU()(self.rpn_conv(feat))
        cls_scores = self.cls_layer(rpn_feat)
        box_transform_pred = self.bbox_reg_layer(rpn_feat)

        # Bước 2: Sinh anchors
        anchors = self.generate_anchors(image, feat)

        # Định hình lại cls_scores thành (Batch_Size * H_feat * W_feat * Số anchors mỗi vị trí, 1)
        number_of_anchors_per_location = cls_scores.size(1)
        cls_scores = cls_scores.permute(0, 2, 3, 1).reshape(-1, 1)

        # Định hình lại box_transform_pred thành (Batch_Size * H_feat * W_feat * Số anchors mỗi vị trí, 4)
        box_transform_pred = box_transform_pred.view(
            box_transform_pred.size(0),
            number_of_anchors_per_location,
            4,
            rpn_feat.shape[-2],
            rpn_feat.shape[-1]
        ).permute(0, 3, 4, 1, 2).reshape(-1, 4)

        # Bước 3: Biến đổi anchors thành proposals dựa trên box_transform_pred
        proposals = apply_regression_pred_to_anchors_or_proposals(
            box_transform_pred.detach().reshape(-1, 1, 4),
            anchors
        )
        proposals = proposals.reshape(proposals.size(0), 4)

        # Bước 4: Lọc proposals
        proposals, scores = self.filter_proposals(proposals, cls_scores.detach(), image.shape)
        rpn_output = {'proposals': proposals, 'scores': scores}

        if not self.training or target is None:
            # Nếu không huấn luyện, trả về proposals và scores
            return rpn_output

        # Bước 5: Gán nhãn ground truth cho từng anchor
        labels_for_anchors, matched_gt_boxes_for_anchors = self.assign_targets_to_anchors(
            anchors, target['bboxes'][0]
        )

        # Tính toán regression targets từ anchors và matched_gt_boxes
        regression_targets = boxes_to_transformation_targets(matched_gt_boxes_for_anchors, anchors)

        # Lấy mẫu anchors dương và âm
        sampled_neg_idx_mask, sampled_pos_idx_mask = sample_positive_negative(
            labels_for_anchors,
            positive_count=self.rpn_pos_count,
            total_count=self.rpn_batch_size
        )

        sampled_idxs = torch.where(sampled_pos_idx_mask | sampled_neg_idx_mask)[0]

        # Tính toán mất mát localization
        localization_loss = (
            torch.nn.functional.smooth_l1_loss(
                box_transform_pred[sampled_pos_idx_mask],
                regression_targets[sampled_pos_idx_mask],
                beta=1 / 9,
                reduction="sum",
            ) / sampled_idxs.numel()
        )

        # Tính toán mất mát phân loại
        cls_loss = torch.nn.functional.binary_cross_entropy_with_logits(
            cls_scores[sampled_idxs].flatten(),
            labels_for_anchors[sampled_idxs].flatten()
        )

        rpn_output['rpn_classification_loss'] = cls_loss
        rpn_output['rpn_localization_loss'] = localization_loss
        return rpn_output



## 4. ROI Head

Tiếp theo, em sẽ xây dựng một **RoI Head** với mục tiêu trích xuất các đặc trưng từ các vùng được đề xuất từ bước **RPN**, làm tiền đề cho phân loại và hồi quy cho bounding box.

Sau khi có các **region proposals** từ **Region Proposal Network (RPN)**, RoI Head sử dụng các RoI này để trích xuất các đặc trưng từ đặc trưng toàn cục của ảnh. Các RoI có thể có kích thước khác nhau, nhưng RoI Head sử dụng kỹ thuật **RoI Pooling** để chuẩn hóa tất cả các RoI về kích thước cố định, giúp dễ dàng xử lý trong các bước tiếp theo.

#### Các phương thức chính trong RoI Head:

1. **assign_target_to_proposals**:
   - Gán nhãn cho các đề xuất (**proposals**) dựa trên sự tương quan với các hộp **ground truth (gt_boxes)**. Mỗi proposal sẽ được phân loại là foreground, background hoặc ignored tùy theo sự tương quan với các hộp ground truth.

2. **filter_predictions**:
   - Lọc ra các proposal, nhưng lần này cần đảm bảo rằng các đề xuất được lọc chính xác để tạo thành đầu ra của mô hình, đảm bảo chất lượng của các dự đoán.

3. **forward**:
   Phương thức chính thực hiện các bước sau:
   1. Nếu đang huấn luyện:
      - Gán các box và nhãn mục tiêu cho tất cả các proposal.
      - Lấy mẫu các proposal dương tính và tiêu cực.
      - Lấy mục tiêu biến đổi bounding box cho tất cả các proposal dựa trên các nhãn được gán.
   2. Lấy các đặc trưng đã qua **RoI Pooling** cho tất cả các proposal.
   3. Gọi các lớp **fully connected (fc6, fc7)** và các lớp phân loại và hồi quy bounding box.
   4. Tính toán lỗi phân loại và lỗi vị trí (localization loss).

#### Các lớp và kỹ thuật trong RoI Head:

- **RoI Pooling**: Kỹ thuật này giúp trích xuất các đặc trưng từ các RoI có kích thước khác nhau và chuẩn hóa chúng về kích thước cố định, từ đó giúp xử lý các RoI trong các bước tiếp theo của mô hình.

- **Fully Connected Layers (fc6, fc7)**: Các lớp fully connected này giúp chuyển đổi các đặc trưng đã qua RoI Pooling thành các đặc trưng cuối cùng, phục vụ cho phân loại và dự đoán bounding box.

- **Loss Calculation**:
   - **Classification Loss**: Được tính bằng Cross-Entropy Loss giữa nhãn thực tế (ground truth) và nhãn dự đoán của các RoI.
   - **Localization Loss**: Được tính bằng **Smooth L1 Loss** hoặc **L2 Loss** giữa các tọa độ thực tế (ground truth bbox) và các tọa độ dự đoán của các RoI.


In [8]:
class ROIHead(nn.Module):
    r"""
    ROI head trên lớp ROI pooling để tạo ra các dự đoán phân loại và biến đổi box.
    Chúng ta có hai lớp fc (fully connected) theo sau là lớp fc phân loại và lớp fc hồi quy bbox.
    """

    
    def __init__(self, model_config, num_classes, in_channels):
        super(ROIHead, self).__init__()
        self.num_classes = num_classes
        self.roi_batch_size = model_config['roi_batch_size']
        self.roi_pos_count = int(model_config['roi_pos_fraction'] * self.roi_batch_size)
        self.iou_threshold = model_config['roi_iou_threshold']
        self.low_bg_iou = model_config['roi_low_bg_iou']
        self.nms_threshold = model_config['roi_nms_threshold']
        self.topK_detections = model_config['roi_topk_detections']
        self.low_score_threshold = model_config['roi_score_threshold']
        self.pool_size = model_config['roi_pool_size']
        self.fc_inner_dim = model_config['fc_inner_dim']
        
        self.fc6 = nn.Linear(in_channels * self.pool_size * self.pool_size, self.fc_inner_dim)
        self.fc7 = nn.Linear(self.fc_inner_dim, self.fc_inner_dim)
        self.cls_layer = nn.Linear(self.fc_inner_dim, self.num_classes)
        self.bbox_reg_layer = nn.Linear(self.fc_inner_dim, self.num_classes * 4)
        
        torch.nn.init.normal_(self.cls_layer.weight, std=0.01)
        torch.nn.init.constant_(self.cls_layer.bias, 0)

        torch.nn.init.normal_(self.bbox_reg_layer.weight, std=0.001)
        torch.nn.init.constant_(self.bbox_reg_layer.bias, 0)
    
    def assign_target_to_proposals(self, proposals, gt_boxes, gt_labels):
        r"""
        Gán các đề xuất (proposals) cho các hộp ground truth (gt_boxes) và nhãn của chúng (gt_labels) 
        dựa trên giá trị IOU (Intersection over Union). 
        Sử dụng IOU để gán các đề xuất cho hộp ground truth hoặc background.
        
        :param proposals: (số lượng đề xuất, 4) - Các hộp đề xuất (proposals) trong không gian ảnh.
        :param gt_boxes: (số lượng hộp ground truth, 4) - Các hộp ground truth (gt_boxes).
        :param gt_labels: (số lượng hộp ground truth) - Các nhãn của các hộp ground truth.
        
        :return:
            - labels: (số lượng đề xuất) - Nhãn được gán cho các đề xuất, bao gồm các nhãn của đối tượng hoặc background.
            - matched_gt_boxes: (số lượng đề xuất, 4) - Các hộp ground truth được gán cho các đề xuất.
        """
        
        # Tính toán ma trận IOU giữa các hộp ground truth và các đề xuất
        iou_matrix = get_iou(gt_boxes, proposals)
        
        # Tìm hộp ground truth phù hợp nhất cho từng đề xuất
        best_match_iou, best_match_gt_idx = iou_matrix.max(dim=0)
        
        # Đánh dấu các đề xuất thuộc về background (IOU thấp hơn threshold)
        background_proposals = (best_match_iou < self.iou_threshold) & (best_match_iou >= self.low_bg_iou)
        
        # Đánh dấu các đề xuất bị bỏ qua (IOU quá thấp)
        ignored_proposals = best_match_iou < self.low_bg_iou
        
        # Cập nhật chỉ mục của các đề xuất thuộc background và bị bỏ qua
        best_match_gt_idx[background_proposals] = -1
        best_match_gt_idx[ignored_proposals] = -2
        
        # Lấy các hộp ground truth phù hợp nhất cho tất cả các đề xuất
        # Các đề xuất background cũng sẽ có một hộp ground truth được gán
        matched_gt_boxes_for_proposals = gt_boxes[best_match_gt_idx.clamp(min=0)]
        
        # Lấy nhãn của các đề xuất theo các hộp ground truth phù hợp
        labels = gt_labels[best_match_gt_idx.clamp(min=0)]
        labels = labels.to(dtype=torch.int64)
        
        # Gán nhãn 0 (background) cho các đề xuất thuộc background
        labels[background_proposals] = 0
        
        # Gán nhãn -1 cho các đề xuất bị bỏ qua (ignored proposals)
        labels[ignored_proposals] = -1
        
        return labels, matched_gt_boxes_for_proposals
        
    def forward(self, feat, proposals, image_shape, target):
        r"""
        Phương thức chính cho ROI head thực hiện các bước sau:
        1. Nếu đang huấn luyện, gán các box và nhãn mục tiêu cho tất cả các proposal.
        2. Nếu đang huấn luyện, lấy mẫu các proposal dương tính và tiêu cực.
        3. Nếu đang huấn luyện, lấy mục tiêu biến đổi bbox cho tất cả các proposal dựa trên các gán nhãn.
        4. Lấy các đặc trưng đã qua ROI Pooling cho tất cả các proposal.
        5. Gọi các lớp fc6, fc7 và các lớp phân loại và biến đổi bbox.
        6. Tính toán lỗi phân loại và lỗi vị trí (localization).

        :param feat: Tensor đặc trưng của hình ảnh đầu vào.
        :param proposals: Các proposal dự đoán, có dạng (số lượng proposal, 4).
        :param image_shape: Kích thước của hình ảnh đầu vào (height, width).
        :param target: Mục tiêu trong quá trình huấn luyện, chứa các hộp gốc và nhãn.
        :return: frcnn_output: Một dictionary chứa các thông tin đầu ra của mô hình, bao gồm:
                - Hệ số phân loại và vị trí trong quá trình huấn luyện.
                - Các hộp, nhãn và điểm số trong quá trình suy luận.
        """
        if self.training and target is not None:
            # Thêm ground truth vào proposals
            proposals = torch.cat([proposals, target['bboxes'][0]], dim=0)
            
            gt_boxes = target['bboxes'][0]
            gt_labels = target['labels'][0]
            
            # Gán mục tiêu cho proposals
            labels, matched_gt_boxes_for_proposals = self.assign_target_to_proposals(proposals, gt_boxes, gt_labels)
            
            # Lấy mẫu các proposal dương tính và tiêu cực
            sampled_neg_idx_mask, sampled_pos_idx_mask = sample_positive_negative(labels,
                                                                                positive_count=self.roi_pos_count,
                                                                                total_count=self.roi_batch_size)
            
            # Chọn các proposal đã được lấy mẫu
            sampled_idxs = torch.where(sampled_pos_idx_mask | sampled_neg_idx_mask)[0]
            
            # Chỉ giữ lại các proposal đã được lấy mẫu
            proposals = proposals[sampled_idxs]
            labels = labels[sampled_idxs]
            matched_gt_boxes_for_proposals = matched_gt_boxes_for_proposals[sampled_idxs]
            
            # Lấy các mục tiêu biến đổi bbox
            regression_targets = boxes_to_transformation_targets(matched_gt_boxes_for_proposals, proposals)
        
        # Tính toán tỷ lệ (scale) cần thiết cho ROI Pooling
        size = feat.shape[-2:]
        possible_scales = []
        for s1, s2 in zip(size, image_shape):
            approx_scale = float(s1) / float(s2)
            scale = 2 ** float(torch.tensor(approx_scale).log2().round())
            possible_scales.append(scale)
        assert possible_scales[0] == possible_scales[1]
        
        # ROI Pooling và gọi các lớp để dự đoán
        proposal_roi_pool_feats = torchvision.ops.roi_pool(feat, [proposals],
                                                        output_size=self.pool_size,
                                                        spatial_scale=possible_scales[0])
        proposal_roi_pool_feats = proposal_roi_pool_feats.flatten(start_dim=1)
        
        # Dự đoán thông qua các lớp fc6 và fc7
        box_fc_6 = torch.nn.functional.relu(self.fc6(proposal_roi_pool_feats))
        box_fc_7 = torch.nn.functional.relu(self.fc7(box_fc_6))
        
        # Tính toán các dự đoán phân loại và biến đổi bbox
        cls_scores = self.cls_layer(box_fc_7)
        box_transform_pred = self.bbox_reg_layer(box_fc_7)
        
        num_boxes, num_classes = cls_scores.shape
        box_transform_pred = box_transform_pred.reshape(num_boxes, num_classes, 4)
        
        frcnn_output = {}
        
        if self.training and target is not None:
            # Tính toán lỗi phân loại
            classification_loss = torch.nn.functional.cross_entropy(cls_scores, labels)
            
            # Tính toán lỗi vị trí chỉ cho các proposals không phải background
            fg_proposals_idxs = torch.where(labels > 0)[0]
            fg_cls_labels = labels[fg_proposals_idxs]
            
            # Lỗi vị trí (localization loss) cho các proposal dương tính
            localization_loss = torch.nn.functional.smooth_l1_loss(
                box_transform_pred[fg_proposals_idxs, fg_cls_labels],
                regression_targets[fg_proposals_idxs],
                beta=1/9,
                reduction="sum",
            )
            localization_loss = localization_loss / labels.numel()
            
            # Lưu kết quả lỗi phân loại và vị trí
            frcnn_output['frcnn_classification_loss'] = classification_loss
            frcnn_output['frcnn_localization_loss'] = localization_loss
        
        if self.training:
            return frcnn_output
        else:
            device = cls_scores.device
            
            # Áp dụng các dự đoán biến đổi bbox lên các proposals
            pred_boxes = apply_regression_pred_to_anchors_or_proposals(box_transform_pred, proposals)
            
            # Áp dụng softmax cho điểm số phân loại
            pred_scores = torch.nn.functional.softmax(cls_scores, dim=-1)
            
            # Clamp hộp dự đoán vào giới hạn của hình ảnh
            pred_boxes = clamp_boxes_to_image_boundary(pred_boxes, image_shape)
            
            # Tạo nhãn cho mỗi dự đoán
            pred_labels = torch.arange(num_classes, device=device)
            pred_labels = pred_labels.view(1, -1).expand_as(pred_scores)
            
            # Loại bỏ các dự đoán có nhãn background
            pred_boxes = pred_boxes[:, 1:]
            pred_scores = pred_scores[:, 1:]
            pred_labels = pred_labels[:, 1:]
            
            # Batch các dự đoán cho mỗi lớp
            pred_boxes = pred_boxes.reshape(-1, 4)
            pred_scores = pred_scores.reshape(-1)
            pred_labels = pred_labels.reshape(-1)
            
            # Lọc các dự đoán sau khi NMS
            pred_boxes, pred_labels, pred_scores = self.filter_predictions(pred_boxes, pred_labels, pred_scores)
            
            # Lưu các kết quả dự đoán
            frcnn_output['boxes'] = pred_boxes
            frcnn_output['scores'] = pred_scores
            frcnn_output['labels'] = pred_labels
            return frcnn_output

    
    def filter_predictions(self, pred_boxes, pred_labels, pred_scores):
        r"""
        Phương thức để lọc các dự đoán (predictions) bằng cách thực hiện các bước sau:
        1. Lọc các hộp có điểm số thấp.
        2. Loại bỏ các hộp có kích thước nhỏ.
        3. Thực hiện NMS (Non-Maximum Suppression) cho từng lớp riêng biệt.
        4. Giữ lại chỉ topK phát hiện.

        :param pred_boxes: (tensor) Các hộp dự đoán, kích thước (num_boxes, 4), mỗi hộp có định dạng [x1, y1, x2, y2].
        :param pred_labels: (tensor) Các nhãn dự đoán, kích thước (num_boxes), chứa các lớp cho mỗi hộp.
        :param pred_scores: (tensor) Các điểm số dự đoán, kích thước (num_boxes), chứa độ tin cậy của mỗi hộp.
        :return: 
            - pred_boxes: (tensor) Các hộp dự đoán sau khi lọc, kích thước (num_filtered_boxes, 4).
            - pred_labels: (tensor) Các nhãn dự đoán tương ứng sau khi lọc.
            - pred_scores: (tensor) Các điểm số dự đoán sau khi lọc.
        """
        
        # Lọc các hộp có điểm số thấp
        keep = torch.where(pred_scores > self.low_score_threshold)[0]
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        
        # Loại bỏ các hộp có kích thước nhỏ
        min_size = 16
        ws, hs = pred_boxes[:, 2] - pred_boxes[:, 0], pred_boxes[:, 3] - pred_boxes[:, 1]
        keep = (ws >= min_size) & (hs >= min_size)
        keep = torch.where(keep)[0]
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        
        # Thực hiện NMS cho từng lớp riêng biệt
        keep_mask = torch.zeros_like(pred_scores, dtype=torch.bool)
        for class_id in torch.unique(pred_labels):
            curr_indices = torch.where(pred_labels == class_id)[0]
            curr_keep_indices = torch.ops.torchvision.nms(pred_boxes[curr_indices],
                                                        pred_scores[curr_indices],
                                                        self.nms_threshold)
            keep_mask[curr_indices[curr_keep_indices]] = True
        
        # Giữ lại các hộp dự đoán sau khi NMS
        keep_indices = torch.where(keep_mask)[0]
        post_nms_keep_indices = keep_indices[pred_scores[keep_indices].sort(descending=True)[1]]
        keep = post_nms_keep_indices[:self.topK_detections]
        
        # Lọc các hộp dự đoán theo chỉ số đã chọn
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        
        return pred_boxes, pred_labels, pred_scores


## 5. Faster RCNN
Cuối cùng xây dựng một faster RCNN hoàn chỉnh bao gồm RPN và RoI head

In [9]:
def transform_boxes_to_original_size(boxes, new_size, original_size):
    """
    Hàm chuyển đổi các hộp chứa (bounding boxes) từ kích thước đã thay đổi (resize) 
    về kích thước gốc của ảnh trước khi thay đổi kích thước.

    Khi ảnh được thay đổi kích thước, các hộp chứa cần được điều chỉnh lại 
    để phù hợp với kích thước gốc của ảnh. Hàm này thực hiện phép toán tỷ lệ 
    giữa kích thước ảnh đã thay đổi và kích thước gốc, sau đó áp dụng tỷ lệ này 
    để điều chỉnh các tọa độ của hộp chứa.

    Args:
        boxes (Tensor): Tensor chứa các hộp chứa sau khi thay đổi kích thước, có dạng (Batchsize x N x 4), 
                         trong đó mỗi hộp chứa có 4 tọa độ [xmin, ymin, xmax, ymax].
        new_size (tuple): Kích thước của ảnh sau khi thay đổi kích thước (new_height, new_width).
        original_size (tuple): Kích thước gốc của ảnh trước khi thay đổi kích thước (original_height, original_width).
    
    Returns:
        Tensor: Tensor chứa các hộp chứa sau khi được chuyển đổi về kích thước gốc của ảnh, có cùng dạng với `boxes`.
    """
    # Tính toán tỷ lệ giữa kích thước gốc và kích thước đã thay đổi
    ratios = [
        torch.tensor(s_orig, dtype=torch.float32, device=boxes.device)
        / torch.tensor(s, dtype=torch.float32, device=boxes.device)
        for s, s_orig in zip(new_size, original_size)
    ]
    ratio_height, ratio_width = ratios

    # Tách các tọa độ xmin, ymin, xmax, ymax của các hộp chứa
    xmin, ymin, xmax, ymax = boxes.unbind(1)

    # Điều chỉnh các tọa độ của hộp chứa theo tỷ lệ đã tính toán
    xmin = xmin * ratio_width
    xmax = xmax * ratio_width
    ymin = ymin * ratio_height
    ymax = ymax * ratio_height

    # Trả lại các hộp chứa đã được điều chỉnh về kích thước gốc
    return torch.stack((xmin, ymin, xmax, ymax), dim=1)


In [10]:
class FasterRCNN(nn.Module):
    def __init__(self, model_config, num_classes, backbone):
        """
        Hàm khởi tạo cho lớp FasterRCNN.

        Args:
            model_config (dict): Cấu hình của mô hình, chứa các thông số như kích thước đầu vào của ảnh, số kênh đầu ra của backbone, các tỉ lệ tỉ lệ của RPN, ...
            num_classes (int): Số lượng lớp cần phân loại (bao gồm cả lớp nền).
            backbone (nn.Module): Mạng trích xuất đặc trưng (ví dụ: VGG16, ResNet, v.v.)
                - input: Batchsize x in_channels x H x W
                - output: Batchsize x out_channels x H_out x W_out (ví dụ: 20 -> 40)
        """
        super(FasterRCNN, self).__init__()

        # Lưu lại cấu hình của mô hình
        self.model_config = model_config
        
        # Backbone sẽ dùng để trích xuất đặc trưng
        self.backbone = backbone

        # Khởi tạo Region Proposal Network (RPN) với các tham số từ cấu hình
        self.rpn = RegionProposalNetwork(
            model_config['backbone_out_channels'],
            scales=model_config['scales'],
            aspect_ratios=model_config['aspect_ratios'],
            model_config=model_config
        )

        # Khởi tạo ROIHead để xử lý các đề xuất (proposals)
        self.roi_head = ROIHead(model_config, num_classes, in_channels=model_config['backbone_out_channels'])

        # Khởi tạo các thông số cho ảnh
        self.image_mean = [0.485, 0.456, 0.406]
        self.image_std = [0.229, 0.224, 0.225]

        # Kích thước tối thiểu và tối đa của ảnh trong quá trình huấn luyện
        self.min_size = model_config['min_im_size']
        self.max_size = model_config['max_im_size']

    
    def normalize_resize_image_and_boxes(self, image, bboxes):
        """
        Hàm chuẩn hóa và thay đổi kích thước ảnh và các hộp chứa (bounding boxes).

        Các bước thực hiện:
        1. Chuẩn hóa ảnh bằng cách trừ đi giá trị trung bình và chia cho độ lệch chuẩn.
        2. Thay đổi kích thước ảnh sao cho chiều nhỏ nhất được thay đổi tới 600, và chiều lớn hơn không vượt quá 1000.
        3. Nếu có, thay đổi kích thước của các hộp chứa tương ứng với kích thước ảnh mới.

        Args:
            image (Tensor): Tensor ảnh đầu vào có dạng (Batchsize x Channels x Height x Width).
            bboxes (Tensor, optional): Tensor chứa các hộp chứa với dạng (Batchsize x N_boxes x 4). 
                                    Mỗi hộp chứa được biểu diễn dưới dạng [xmin, ymin, xmax, ymax].

        Returns:
            image (Tensor): Ảnh sau khi chuẩn hóa và thay đổi kích thước.
            bboxes (Tensor, optional): Các hộp chứa sau khi thay đổi kích thước, nếu có.
        """
        dtype, device = image.dtype, image.device
        
        # Chuẩn hóa ảnh
        mean = torch.as_tensor(self.image_mean, dtype=dtype, device=device)
        std = torch.as_tensor(self.image_std, dtype=dtype, device=device)
        image = (image - mean[:, None, None]) / std[:, None, None]

        # Thay đổi kích thước ảnh sao cho chiều nhỏ nhất được thay đổi tới 600, và chiều lớn hơn không vượt quá 1000
        h, w = image.shape[-2:]
        im_shape = torch.tensor(image.shape[-2:])
        min_size = torch.min(im_shape).to(dtype=torch.float32)
        max_size = torch.max(im_shape).to(dtype=torch.float32)
        
        # Tính tỷ lệ thay đổi kích thước sao cho chiều nhỏ nhất là 600 và chiều lớn nhất không vượt quá 1000
        scale = torch.min(float(self.min_size) / min_size, float(self.max_size) / max_size)
        scale_factor = scale.item()

        # Thay đổi kích thước ảnh dựa trên tỷ lệ đã tính toán
        image = torch.nn.functional.interpolate(
            image,
            size=None,
            scale_factor=scale_factor,
            mode="bilinear",
            recompute_scale_factor=True,
            align_corners=False,
        )

        if bboxes is not None:
            # Thay đổi kích thước các hộp chứa (bboxes) tương ứng với tỷ lệ thay đổi kích thước của ảnh
            ratios = [
                torch.tensor(s, dtype=torch.float32, device=bboxes.device)
                / torch.tensor(s_orig, dtype=torch.float32, device=bboxes.device)
                for s, s_orig in zip(image.shape[-2:], (h, w))
            ]
            ratio_height, ratio_width = ratios
            
            # Cập nhật vị trí của các hộp chứa
            xmin, ymin, xmax, ymax = bboxes.unbind(2)
            xmin = xmin * ratio_width
            xmax = xmax * ratio_width
            ymin = ymin * ratio_height
            ymax = ymax * ratio_height
            bboxes = torch.stack((xmin, ymin, xmax, ymax), dim=2)

        return image, bboxes

    
    def forward(self, image, target=None):
        """
        Hàm thực hiện bước truyền qua (forward pass) trong mô hình Faster R-CNN.

        Các bước thực hiện:
        1. Chuẩn hóa và thay đổi kích thước ảnh và các hộp chứa (bounding boxes) trong trường hợp huấn luyện.
        2. Tiến hành trích xuất đặc trưng từ ảnh thông qua mạng xương sống (backbone).
        3. Sử dụng Mạng Đề xuất Vùng (RPN) để sinh ra các đề xuất (proposals).
        4. Sử dụng ROI head để biến các đề xuất thành các hộp chứa (bounding boxes) cuối cùng.
        5. Nếu không phải trong quá trình huấn luyện, trả lại các hộp chứa đã được chuyển đổi về kích thước ảnh gốc.

        Args:
            image (Tensor): Tensor ảnh đầu vào có dạng (Batchsize x Channels x Height x Width).
            target (dict, optional): Một từ điển chứa thông tin về các hộp chứa (bboxes) cho mỗi đối tượng trong ảnh, 
                                    được sử dụng trong quá trình huấn luyện. Từ điển có thể chứa:
                                    - 'bboxes': các hộp chứa trong ảnh có dạng [xmin, ymin, xmax, ymax].
                                    
        Returns:
            rpn_output (dict): Kết quả từ Mạng Đề xuất Vùng (RPN), bao gồm các đề xuất (proposals).
            frcnn_output (dict): Kết quả từ ROI head, bao gồm các hộp chứa cuối cùng và các thông tin liên quan.
        """
        old_shape = image.shape[-2:]

        if self.training:
            # Chuẩn hóa và thay đổi kích thước ảnh và các hộp chứa trong trường hợp huấn luyện
            image, bboxes = self.normalize_resize_image_and_boxes(image, target['bboxes'])
            target['bboxes'] = bboxes
        else:
            # Chỉ chuẩn hóa và thay đổi kích thước ảnh mà không có hộp chứa trong trường hợp suy luận (inference)
            image, _ = self.normalize_resize_image_and_boxes(image, None)
        
        # Trích xuất đặc trưng từ ảnh thông qua backbone
        feat = self.backbone(image)
        
        # Tiến hành sinh các đề xuất (proposals) từ RPN
        rpn_output = self.rpn(image, feat, target)
        proposals = rpn_output['proposals']
        
        # Biến các đề xuất thành các hộp chứa cuối cùng thông qua ROI head
        frcnn_output = self.roi_head(feat, proposals, image.shape[-2:], target)
        
        if not self.training:
            # Chuyển đổi các hộp chứa về kích thước gốc của ảnh, chỉ được gọi trong quá trình suy luận
            frcnn_output['boxes'] = transform_boxes_to_original_size(frcnn_output['boxes'],
                                                                    image.shape[-2:],
                                                                    old_shape)
        
        return rpn_output, frcnn_output


# Xây dựng Dataset, Dataloader
Sau khi hoàn thành việc thiết lập model, tiếp theo em sẽ thực hiện việc lấy dữ liệu
đóng gói dữ liệu thành Dataset, Dataloader


In [11]:
import torch
import torch.nn as nn
from PIL import Image
import cv2

import os
from tqdm import tqdm

from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split
import torchvision

def parse_txt_to_dict(file_path):
    result = {'labels': [], 'bboxes': []}
    
    with open(file_path, 'r') as file:
        for line in file:
            # Tách các giá trị trong dòng
            values = line.split()
            label = int(values[0])  # Nhãn
            bbox = list(map(float, values[1:]))  # Hộp giới hạn (bounding box)
            
            # Thêm vào kết quả
            result['labels'].append(label)
            result['bboxes'].append(bbox)
    
    return result

def get_valid_file_in_folder(folder_path, valid_extensions):
    valid_files_path = []

    for file_name in os.listdir(folder_path):
        file_extension = os.path.splitext(file_name)[1].lower()  # Lấy phần mở rộng
        if file_extension in valid_extensions:
            valid_files_path.append(os.path.join(folder_path, file_name))

    return valid_files_path

def load_images_and_labels(folder_path):

    # Lấy các file images
    valid_extensions_image = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"}
    folder_contain_images_path = os.path.join(folder_path, 'images')
    image_paths = get_valid_file_in_folder(folder_contain_images_path, valid_extensions_image)

    # Lấy các file labels
    folder_contain_labels_path = os.path.join(folder_path, 'labels')
    label_paths = []
    for file_name in image_paths:
        label_file_name = os.path.splitext(os.path.basename(file_name))[0] + '.txt'
        label_paths.append(os.path.join(folder_contain_labels_path, label_file_name))
    
    return list(zip(image_paths, label_paths))
    


class ObjectDetectionDataset(Dataset):
    def __init__(self, pair_path_image_label):
        # self.data = load_images_and_labels(folder_path)
        self.data = pair_path_image_label
        classes = [
            'Apple', 'Grapes', 'Pineapple', 'Orange', 'Banana', 'Watermelon'
        ]
        self.label2idx = {classes[idx]: idx for idx in range(len(classes))}
        self.idx2label = {idx: classes[idx] for idx in range(len(classes))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path, target_path = self.data[index]

        image = Image.open(image_path)
        target = parse_txt_to_dict(target_path)

        image_tensor = torchvision.transforms.ToTensor()(image)
        target['labels'] = torch.as_tensor(target['labels'], dtype=torch.int64)
        target['bboxes'] = torch.as_tensor(target['bboxes'], dtype=torch.float32)

        return image_tensor, target, torch.as_tensor(np.array(image))


import random

test_size = 20
folder_path = '/kaggle/input/k-67-object-detection/Dataset/Train'
pair_path_image_label = load_images_and_labels(folder_path)

random.shuffle(pair_path_image_label)

train_pair_path_image_label = pair_path_image_label[:-test_size]
test_pair_path_image_label = pair_path_image_label[-test_size:]

# dataset = ObjectDetectionDataset(folder_path)

train_dataset = ObjectDetectionDataset(train_pair_path_image_label)
test_dataset = ObjectDetectionDataset(test_pair_path_image_label)

# Traning

In [12]:
config = {
    "model_params": {
        "im_channels": 3,
        "aspect_ratios": [0.5, 1, 2],
        "scales": [128, 256, 512],
        "min_im_size": 600,
        "max_im_size": 1000,
        "backbone_out_channels": 512,
        "fc_inner_dim": 1024,
        "rpn_bg_threshold": 0.3,
        "rpn_fg_threshold": 0.7,
        "rpn_nms_threshold": 0.7,
        "rpn_train_prenms_topk": 12000,
        "rpn_test_prenms_topk": 6000,
        "rpn_train_topk": 2000,
        "rpn_test_topk": 300,
        "rpn_batch_size": 256,
        "rpn_pos_fraction": 0.5,
        "roi_iou_threshold": 0.5,
        "roi_low_bg_iou": 0.0,  
        "roi_pool_size": 7,
        "roi_nms_threshold": 0.3,
        "roi_topk_detections": 100,
        "roi_score_threshold": 0.05,
        "roi_batch_size": 128,
        "roi_pos_fraction": 0.25,
    },
    "train_params": {
        "task_name": "fruit_detection",
        "seed": 1111,
        "acc_steps": 1,
        "num_epochs": 40, # 20
        "lr_steps": [12, 16],
        "lr": 0.001,
        "ckpt_name": "faster_rcnn_fruit_detection.pth",
        "name_backbone": "default"
    },
    "dataset_params": {
        'num_classes' : 6
    }
}

dataset_config = config['dataset_params']
model_config = config['model_params']
train_config = config['train_params']

In [13]:
import torch
import os
import numpy as np
import random
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
from torch.optim.lr_scheduler import MultiStepLR


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train(model_config, train_config, dataset_config, backbone):
    
    # dataset_config = config['dataset_params']
    # model_config = config['model_params']
    # train_config = config['train_params']
    
    seed = train_config['seed']
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(seed)
    
    
    train_dataloader = DataLoader(
        train_dataset, # train_dataset
        batch_size=1,
        shuffle=True,
        num_workers=4
    )
    
    faster_rcnn_model = FasterRCNN(
        model_config, 
        num_classes=dataset_config['num_classes'],
        backbone=backbone
    )
    faster_rcnn_model.train()
    faster_rcnn_model.to(device)

    if not os.path.exists(train_config['task_name']):
        os.mkdir(train_config['task_name'])

    optimizer = torch.optim.SGD(lr=train_config['lr'],
                                params=filter(lambda p: p.requires_grad,
                                              faster_rcnn_model.parameters()),
                                weight_decay=5E-4,
                                momentum=0.9)
    scheduler = MultiStepLR(optimizer, milestones=train_config['lr_steps'], gamma=0.1)
    
    acc_steps = train_config['acc_steps']
    num_epochs = train_config['num_epochs']
    step_count = 1

    for i in range(num_epochs):
        rpn_classification_losses = []
        rpn_localization_losses = []
        frcnn_classification_losses = []
        frcnn_localization_losses = []
        optimizer.zero_grad()
        
        for im, target, _ in tqdm(train_dataloader):
            im = im.float().to(device)
            target['bboxes'] = target['bboxes'].float().to(device)
            target['labels'] = target['labels'].long().to(device)
            rpn_output, frcnn_output = faster_rcnn_model(im, target)
            
            rpn_loss = rpn_output['rpn_classification_loss'] + rpn_output['rpn_localization_loss']
            frcnn_loss = frcnn_output['frcnn_classification_loss'] + frcnn_output['frcnn_localization_loss']
            loss = rpn_loss + frcnn_loss
            
            rpn_classification_losses.append(rpn_output['rpn_classification_loss'].item())
            rpn_localization_losses.append(rpn_output['rpn_localization_loss'].item())
            frcnn_classification_losses.append(frcnn_output['frcnn_classification_loss'].item())
            frcnn_localization_losses.append(frcnn_output['frcnn_localization_loss'].item())
            loss = loss / acc_steps
            loss.backward()
            if step_count % acc_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            step_count += 1
        print('Finished epoch {}'.format(i))
        optimizer.step()
        optimizer.zero_grad()

        path_save = os.path.join(train_config['task_name'], train_config['name_backbone'], train_config['ckpt_name'])
        dir_save = os.path.dirname(path_save)
        if not os.path.exists(dir_save):
            os.makedirs(dir_save, exist_ok=True)
        
        torch.save(faster_rcnn_model.state_dict(), path_save)
        loss_output = ''
        loss_output += 'RPN Classification Loss : {:.4f}'.format(np.mean(rpn_classification_losses))
        loss_output += ' | RPN Localization Loss : {:.4f}'.format(np.mean(rpn_localization_losses))
        loss_output += ' | FRCNN Classification Loss : {:.4f}'.format(np.mean(frcnn_classification_losses))
        loss_output += ' | FRCNN Localization Loss : {:.4f}'.format(np.mean(frcnn_localization_losses))
        print(loss_output)
        scheduler.step()
    print('Train Completed!')
# train()

In [14]:
backbone_resnet18_default = BackBoneResNet18()
train_config['name_backbone'] = "resnet18_default"
train(model_config, train_config, dataset_config, backbone_resnet18_default)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 202MB/s]
100%|██████████| 7611/7611 [04:33<00:00, 27.78it/s]


Finished epoch 0
RPN Classification Loss : 0.1712 | RPN Localization Loss : 0.0789 | FRCNN Classification Loss : 0.3376 | FRCNN Localization Loss : 0.0447


100%|██████████| 7611/7611 [04:26<00:00, 28.54it/s]


Finished epoch 1
RPN Classification Loss : 0.1103 | RPN Localization Loss : 0.0649 | FRCNN Classification Loss : 0.2152 | FRCNN Localization Loss : 0.0434


100%|██████████| 7611/7611 [04:27<00:00, 28.42it/s]


...

Finished epoch 37
RPN Classification Loss : 0.0068 | RPN Localization Loss : 0.0180 | FRCNN Classification Loss : 0.0395 | FRCNN Localization Loss : 0.0160


100%|██████████| 7611/7611 [04:27<00:00, 28.41it/s]


Finished epoch 38
RPN Classification Loss : 0.0067 | RPN Localization Loss : 0.0180 | FRCNN Classification Loss : 0.0398 | FRCNN Localization Loss : 0.0159


100%|██████████| 7611/7611 [04:27<00:00, 28.40it/s]


Finished epoch 39
RPN Classification Loss : 0.0067 | RPN Localization Loss : 0.0180 | FRCNN Classification Loss : 0.0390 | FRCNN Localization Loss : 0.0159
Train Completed!


# Thử với mô hình Backbone Retnet18, nhưng thêm 1 block với 2 lớp conv2d
Kiến trúc này có khả năng làm giảm kích thước của feature map và tăng trường nhìn


In [15]:
import torch
import torch.nn as nn
import torchvision.models as models
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(512)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(512)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        return x
    
class BackBoneResNet18Add1Block(nn.Module):
    def __init__(self):
        super().__init__()

        weights = models.ResNet18_Weights.DEFAULT
        resnet18 = models.resnet18(weights=weights)

        children = [child for child in resnet18.children()]

        self.model = nn.Sequential(
            *children[:-2], ConvBlock()
        )
        # self.model = resnet18.features[:-1]

    def forward(self, X):

        out = self.model(X)

        return out


In [16]:
backbone_resnet18_add1block = BackBoneResNet18Add1Block()
train_config['name_backbone'] = 'resnet18_add1block'
train(model_config, train_config, dataset_config, backbone_resnet18_add1block)

100%|██████████| 7611/7611 [04:36<00:00, 27.56it/s]


Finished epoch 0
RPN Classification Loss : 0.1550 | RPN Localization Loss : 0.0558 | FRCNN Classification Loss : 0.3804 | FRCNN Localization Loss : 0.0381


100%|██████████| 7611/7611 [04:35<00:00, 27.67it/s]


Finished epoch 1
RPN Classification Loss : 0.0953 | RPN Localization Loss : 0.0462 | FRCNN Classification Loss : 0.2584 | FRCNN Localization Loss : 0.0364


100%|██████████| 7611/7611 [04:35<00:00, 27.67it/s]


...

Finished epoch 37
RPN Classification Loss : 0.0074 | RPN Localization Loss : 0.0141 | FRCNN Classification Loss : 0.0862 | FRCNN Localization Loss : 0.0151


100%|██████████| 7611/7611 [04:35<00:00, 27.65it/s]


Finished epoch 38
RPN Classification Loss : 0.0074 | RPN Localization Loss : 0.0140 | FRCNN Classification Loss : 0.0860 | FRCNN Localization Loss : 0.0151


100%|██████████| 7611/7611 [04:34<00:00, 27.68it/s]


Finished epoch 39
RPN Classification Loss : 0.0073 | RPN Localization Loss : 0.0140 | FRCNN Classification Loss : 0.0860 | FRCNN Localization Loss : 0.0150
Train Completed!


# Inference

In [17]:
import random
import sys

def infer(args):
    if not os.path.exists(f"samples/{args['name_model']}"):
        os.makedirs(f"samples/{args['name_model']}")
    faster_rcnn_model = args['model']
    test_dataset = args['test_dataset']
    
    faster_rcnn_model.roi_head.low_score_threshold = 0.7
    
    for sample_count in tqdm(range(len(test_dataset))):
        im, target, frame = test_dataset[sample_count]
        im = im.unsqueeze(0).float().to(device)
        frame = frame.cpu().numpy()
        # print(frame)

        gt_im = frame.copy()
        gt_im_copy = gt_im.copy()
        
        # Saving images with ground truth boxes
        for idx, box in enumerate(target['bboxes']):
            x1, y1, x2, y2 = box.detach().cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            cv2.rectangle(gt_im, (x1, y1), (x2, y2), thickness=2, color=[0, 255, 0])
            cv2.rectangle(gt_im_copy, (x1, y1), (x2, y2), thickness=2, color=[0, 255, 0])
            text = test_dataset.idx2label[target['labels'][idx].detach().cpu().item()]
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
            text_w, text_h = text_size
            cv2.rectangle(gt_im_copy , (x1, y1), (x1 + 10+text_w, y1 + 10+text_h), [255, 255, 255], -1)
            cv2.putText(gt_im, text=test_dataset.idx2label[target['labels'][idx].detach().cpu().item()],
                        org=(x1+5, y1+15),
                        thickness=1,
                        fontScale=1,
                        color=[0, 0, 0],
                        fontFace=cv2.FONT_HERSHEY_PLAIN)
            cv2.putText(gt_im_copy, text=text,
                        org=(x1 + 5, y1 + 15),
                        thickness=1,
                        fontScale=1,
                        color=[0, 0, 0],
                        fontFace=cv2.FONT_HERSHEY_PLAIN)
        cv2.addWeighted(gt_im_copy, 0.7, gt_im, 0.3, 0, gt_im)
        path_gt = os.path.join('sample', f"{args['name_model']}", 'gt', 'output_frcnn_gt_{}.png'.format(sample_count))
        dir_gt = os.path.dirname(path_gt)
        if not os.path.exists(dir_gt):
            os.makedirs(dir_gt, exist_ok=True)

        cv2.imwrite(path_gt, gt_im)
        
        # Getting predictions from trained model
        rpn_output, frcnn_output = faster_rcnn_model(im, None)
        boxes = frcnn_output['boxes']
        labels = frcnn_output['labels']
        scores = frcnn_output['scores']
        im = frame.copy()
        im_copy = im.copy()
        
        # Saving images with predicted boxes
        for idx, box in enumerate(boxes):
            x1, y1, x2, y2 = box.detach().cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(im, (x1, y1), (x2, y2), thickness=2, color=[0, 0, 255])
            cv2.rectangle(im_copy, (x1, y1), (x2, y2), thickness=2, color=[0, 0, 255])
            text = '{} : {:.2f}'.format(test_dataset.idx2label[labels[idx].detach().cpu().item()],
                                        scores[idx].detach().cpu().item())
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
            text_w, text_h = text_size
            cv2.rectangle(im_copy , (x1, y1), (x1 + 10+text_w, y1 + 10+text_h), [255, 255, 255], -1)
            cv2.putText(im, text=text,
                        org=(x1+5, y1+15),
                        thickness=1,
                        fontScale=1,
                        color=[0, 0, 0],
                        fontFace=cv2.FONT_HERSHEY_PLAIN)
            cv2.putText(im_copy, text=text,
                        org=(x1 + 5, y1 + 15),
                        thickness=1,
                        fontScale=1,
                        color=[0, 0, 0],
                        fontFace=cv2.FONT_HERSHEY_PLAIN)
        cv2.addWeighted(im_copy, 0.7, im, 0.3, 0, im)
        path_pred = os.path.join('sample', f"{args['name_model']}", 'pred', 'output_frcnn_pred_{}.png'.format(sample_count))
        dir_pred = os.path.dirname(path_pred)
        if not os.path.exists(dir_pred):
            os.makedirs(dir_pred, exist_ok=True)
        cv2.imwrite(path_pred.format(sample_count), im)

# infer(cfg_infer)

## Chạy infer, kết quả các ảnh được lưu ở output sample

In [18]:
def run_infer(path_model, backbone, name_model):
    # name_faster_rcnn_model_backbone_resnet18 = '/kaggle/input/trained-model-obj-detection/faster_rcnn_fruit_detection.pth'
    faster_rcnn_model= FasterRCNN(
        model_config, 
        num_classes=dataset_config['num_classes'],
        backbone=backbone
    )
    faster_rcnn_model.load_state_dict(torch.load(path_model, weights_only=True, map_location=device))
    faster_rcnn_model.to(device)
    faster_rcnn_model.eval()
    cfg_infer_faster_rcnn_model = {
        'model': faster_rcnn_model,
        'test_dataset': test_dataset,
        'name_model': name_model
    }
    infer(cfg_infer_faster_rcnn_model)
    
name_faster_rcnn_model_backbone_resnet18 = '/kaggle/working/fruit_detection/resnet18_default/faster_rcnn_fruit_detection.pth'
name_faster_rcnn_model_backbone_resnet18_add1block = '/kaggle/working/fruit_detection/resnet18_add1block/faster_rcnn_fruit_detection.pth'

backbone_noaddblock = BackBoneResNet18()
backbone_add1block = BackBoneResNet18Add1Block()

name_model_noaddblock = 'noaddblock'
name_model_add1block = 'add1block'

run_infer(name_faster_rcnn_model_backbone_resnet18, backbone_noaddblock, name_model_noaddblock)
run_infer(name_faster_rcnn_model_backbone_resnet18_add1block, backbone_add1block, name_model_add1block)
print('Infer completed!')


100%|██████████| 20/20 [00:01<00:00, 13.27it/s]

Infer completed!


# Lưu kết quả khi thực hiện infer trên tập test

In [19]:
import json
import pandas as pd
def create_and_save_results_infering_with_test(model, name_model):
    valid_extensions_image = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"}
    folder_contain_images_path = '/kaggle/input/k-67-object-detection/Dataset/Test/images'
    image_paths = get_valid_file_in_folder(folder_contain_images_path, valid_extensions_image)
    model.roi_head.low_score_threshold = 0.7

    results = {
        'ID': [],
        'bounding_boxes': []
    }

    for image_path in tqdm(image_paths):
        image = Image.open(image_path)
        im_tensor = torchvision.transforms.ToTensor()(image).unsqueeze(0).to(device)

        rpn_output, frcnn_output = model(im_tensor, None)
        boxes = frcnn_output['boxes']
        labels = frcnn_output['labels']
        scores = frcnn_output['scores']

        preds = []
        
        for idx, box in enumerate(boxes):
            x1, y1, x2, y2 = box.detach().cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            label = labels[idx].detach().cpu().item()
            score = scores[idx].detach().cpu().item()

            preds.append({
                'x_min': x1, 'y_min': y1, 'x_max': x2, 'y_max': y2,
                'class': label,
                'confidence': score
            })
        
        file_name_image = os.path.splitext(os.path.basename(image_path))[0]
        results['ID'].append(file_name_image)
        results['bounding_boxes'].append(json.dumps(preds))

    path_save = f"submit/{name_model}/results_objects_detection.csv"
    dir_save = os.path.dirname(path_save)
    if not os.path.exists(dir_save):
        os.makedirs(dir_save, exist_ok=True)

    df = pd.DataFrame(results)

    # Lưu vào file CSV
    df.to_csv(path_save, index=False)
    print('Save completed!')
        
    return results

In [20]:
def run_save_csv(path_model, backbone, name_model):
    faster_rcnn_model= FasterRCNN(
        model_config, 
        num_classes=dataset_config['num_classes'],
        backbone=backbone
    )
    faster_rcnn_model.load_state_dict(torch.load(path_model, weights_only=True, map_location=device))
    faster_rcnn_model.to(device)
    faster_rcnn_model.eval()

    create_and_save_results_infering_with_test(faster_rcnn_model, name_model)

run_save_csv(name_faster_rcnn_model_backbone_resnet18, backbone_noaddblock, name_model_noaddblock)
run_save_csv(name_faster_rcnn_model_backbone_resnet18_add1block, backbone_add1block, name_model_add1block)

100%|██████████| 848/848 [00:34<00:00, 24.83it/s]


Save completed!


100%|██████████| 848/848 [00:26<00:00, 31.60it/s]

Save completed!
